# Service Team Routing
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

The ability of service providers to set service time windows allows for easier and more dependable coordination between the service provider and their customers, while increasing overall customer satisfaction.

In this scenario we have a number of service order locations with associated time windows and service times (time on-site to complete service). Each technician has an associated availability, ability to complete certain types of service, and a maximum number of service appointments per day.

### Problem Details:
- 8 Locations each with an associated demand
    - 1 Headquarters 
        - service type 1 demand: [0]
        - service type 2 demand: [1]
        - headquarters hours of operation: [5,20]
    - 7 Service Locations
        - service type 1 demand: [1, 1, 1, 0, 0, 0, 0]
        - service type 2 demand: [0, 0, 1, 1, 1, 1, 1]
        - service locations time windows: [[9,12],[9,12],[11,14],[13,16],[13,16],[13,16],[13,16]]
        - service location service times: [ 1, 1, 1.5, 0.5, 0.5, 0.5]

- 3 Delivery vehicles each with an associated capacity
    - 3 service technicians
        - capacity for service type 1: [2, 1, 0]
        - capacity for service type 2: [0, 1, 4]
        - technician availability [[9,17], [12,15], [9,17]]
        

### Environment Setup
First, let's check if we have a GPU available in system. This is only applicable if you are running server through this notebook. Else comment the next cell.

In [ ]:
# Check for GPU
!nvidia-smi

#### Install Dependencies

In [2]:
# Install cuOpt

# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

# This would be incase underlying system is cuda-10.X 
# !pip install --user cuopt-server-cu11==25.5.* cuopt-sh-client==25.5.*

# This would be incase underlying system is cuda-12.x
# !pip install --user cuopt-server-cu12==25.5.* cuopt-sh-client==25.5.*

In [3]:
#Install notebook dependencies
!pip install --user -q matplotlib scipy pandas numpy

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from cuopt_sh_client import CuOptServiceSelfHostClient

#### Run cuOpt server in background

Please disable the next cell in case you are trying to test against a server running on a different machine.

In [ ]:
import subprocess
import atexit
import signal
import os
import psutil

def is_cuopt_server_running():
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            if 'python' in proc.info['name'] and \
               any('cuopt_server.cuopt_service' in cmd for cmd in proc.info['cmdline'] if cmd):
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return False

# Only start server if it's not already running
if not is_cuopt_server_running():
    # Start cuOpt server in background
    server_process = subprocess.Popen(['python', '-m', 'cuopt_server.cuopt_service'],
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE)
    
    def cleanup_server():
        try:
            server_process.terminate()
            server_process.wait(timeout=5)  # Wait up to 5 seconds for graceful termination
            print("Terminated cuOpt server")
        except (subprocess.TimeoutExpired, ProcessLookupError):
            # If graceful termination fails, force kill
            try:
                server_process.kill()
                print("Killed cuOpt server")
            except ProcessLookupError:
                pass

    atexit.register(cleanup_server)
    print("Started cuOpt server")
else:
    print("cuOpt server is already running")

#### Utility functions

In [6]:
# Used to plot the Co-ordinates
def gen_plot(df):
    plt.figure(figsize=(11, 11))
    plt.scatter(
        df["xcord"][:1],
        df["ycord"][:1],
        label="Depot",
        color="Green",
        marker="o",
        s=100,
    )
    plt.scatter(
        df["xcord"][1::],
        df["ycord"][1::],
        label="Locations",
        color="Red",
        marker="o",
        s=100,
    )
    plt.xlabel("x - axis")
    # frequency label
    plt.ylabel("y - axis")
    # plot title
    plt.title("Simplified Map")
    # showing legend
    plt.legend()

    for i, label in enumerate(df.index.values):
        plt.annotate(
            label,
            (df["xcord"][i], df["ycord"][i]),
            fontproperties=fm.FontProperties(size=16),
        )
    return plt


# Used to plot arrows
def add_arrows(df, route, plt, color="green"):
    prev_cord = ()
    for i, label in enumerate(route):
        if i > 0:
            plt.annotate(
                "",
                xy=(df["xcord"][label], df["ycord"][label]),
                xytext=prev_cord,
                arrowprops=dict(
                    arrowstyle="simple, head_length=0.5, head_width=0.5, tail_width=0.15",  # noqa
                    connectionstyle="arc3",
                    color=color,
                    mutation_scale=20,
                    ec="black",
                ),
                label="vehicle-1",
            )
        prev_cord = df["xcord"][label], df["ycord"][label]

    return plt


# Convert the solver response from the server to a cuDF dataframe
# for waypoint graph problems
def get_solution_df(resp):
    solution = resp["vehicle_data"]

    df = {}
    df["route"] = []
    df["truck_id"] = []
    df["location"] = []
    types = []

    for vid, route in solution.items():
        df["location"] = df["location"] + route["route"]
        df["truck_id"] = df["truck_id"] + [vid] * len(route["route"])
        if "type" in list(route.keys()):
            types = types + route["type"]
    if len(types) != 0:
        df["types"] = types
    df["route"] = df["location"]

    return pd.DataFrame(df)


# Prints vehicle routes
def show_vehicle_routes(resp, locations):

    solution = resp["vehicle_data"]
    for id in list(solution.keys()):
        route = solution[id]["route"]
        print("For vehicle -", id, "route is: \n")
        path = ""
        for index, route_id in enumerate(route):
            path += locations[route_id]
            if index != (len(route) - 1):
                path += "->"
        print(path + "\n\n")


# Map vehicle routes
def map_vehicle_routes(df, resp, colors):

    solution = resp["vehicle_data"]

    plt = gen_plot(df)
    veh_ids = list(solution.keys())
    idx = 0
    vid_map = {}
    for v_id in veh_ids:
        vid_map[v_id] = idx
        idx = idx + 1

    for v_id in veh_ids:
        plt = add_arrows(
            df, solution[v_id]["route"], plt, color=colors[vid_map[v_id]]
        )

    return plt

Below we visualize the service locations with respect to the service company headquarters. The cost from all locations to all other locations (a cost matrix) will be required for optimization. For the purpose of this simple example we will omit the cost matrix calculation.

In [ ]:
location_names       = [ "Headquarters",     "A",    "B",    "C",    "D",    "E",    "F",    "G"  ]
location_coordinates = [     [4, 4],        [1, 3], [8, 1], [2, 1], [6, 7], [0, 2], [7, 6], [5, 3] ]
location_coordinates_df = pd.DataFrame(location_coordinates, columns=['xcord', 'ycord'], index=location_names)
gen_plot(location_coordinates_df).show()

### Cost Matrix

The cost matrix dictates the cost of travel between locations of interest.  The cost itself can be anything relevant to the user.  In this case we are constraining time window constraints.  When constraining time windows for locations or vehicles it is assumed (if only a single cost matrix is provided) that it represents time. 

Here is the cost(time) matrix corresponding to the locations above:

In [ ]:
time_matrix = [
 [0.00, 0.31, 0.50, 0.36, 0.36, 0.44, 0.36, 0.14],
 [0.31, 0.00, 0.72, 0.22, 0.64, 0.14, 0.67, 0.40],
 [0.50, 0.72, 0.00, 0.60, 0.63, 0.80, 0.51, 0.36],
 [0.36, 0.22, 0.60, 0.00, 0.72, 0.22, 0.70, 0.36],
 [0.36, 0.64, 0.63, 0.72, 0.00, 0.77, 0.14, 0.41],
 [0.44, 0.14, 0.80, 0.22, 0.77, 0.00, 0.80, 0.51],
 [0.36, 0.67, 0.51, 0.70, 0.14, 0.80, 0.00, 0.36],
 [0.14, 0.40, 0.36, 0.36, 0.41, 0.51, 0.36, 0.00]
]

# Create a dataframe of this matrix
time_matrix_df  = pd.DataFrame(time_matrix, 
                              index=location_coordinates_df.index, 
                              columns=location_coordinates_df.index)
time_matrix_df

### Service Locations

Setup the service location data

In [ ]:
# exclude head quarters from service location names
service_location_ids = [1, 2, 3, 4, 5, 6, 7]
service_location_names = [location_names[i] for i in service_location_ids]
service_location_data = {
    "service_location_names": service_location_names,
    "service_location_ids": service_location_ids,
    "service_type1_demand": [1, 1, 1, 0, 0, 0, 0],
    "service_type2_demand": [0, 0, 1, 1, 1, 1, 1],
    "location_earliest_time": [9, 9, 11, 13, 13, 13, 13],
    "location_latest_time": [12, 12, 14, 16, 16, 16,16],
    "required_service_time": [1, 1, 1, 0, 0, 0, 0]
}
service_location_data_df = pd.DataFrame(service_location_data).set_index('service_location_names')
service_location_data_df

### Vehicles

Setup vehicle/technician data

In [ ]:
n_vehicles = 3
vehicle_data = {
    "vehicle_ids": [i for i in range(n_vehicles)],
    "capacity_service_type1":[2, 1, 0],
    "capacity_service_type2":[0, 1, 4],
    "vehicle_availability_earliest":[9, 11, 9],
    "vehicle_availability_latest":[17, 15, 17]
}
vehicle_data_df = pd.DataFrame(vehicle_data).set_index('vehicle_ids')
vehicle_data_df

# Initialize cuOpt Service Client and cuOpt Problem Data

In [11]:
ip = "0.0.0.0"
port = 5000

client = CuOptServiceSelfHostClient(
    ip=ip,
    port=port,
    polling_timeout=None
)

cuopt_problem_data = {}

### Set Cost Matrix

In [12]:
cuopt_problem_data["cost_matrix_data"] = {
        "data": {
            "0": time_matrix
        }
    }

### Set Fleet Data

In [13]:
cuopt_problem_data["fleet_data"] = {
        "vehicle_locations": [[0,0]] * n_vehicles,
        "capacities": [vehicle_data["capacity_service_type1"], vehicle_data["capacity_service_type2"]],
        "vehicle_time_windows": list(zip(vehicle_data['vehicle_availability_earliest'],
                                         vehicle_data['vehicle_availability_latest']))
}

### Set Task Data

In [14]:
cuopt_problem_data["task_data"] = {
        "task_locations": service_location_ids,
        "demand": [service_location_data["service_type1_demand"], 
                   service_location_data["service_type2_demand"]],
        "task_time_windows": list(zip(service_location_data["location_earliest_time"],
                                          service_location_data["location_latest_time"])),
        "service_times": service_location_data["required_service_time"]
}

### Set Solver Configuration

In [15]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }

### Get Optimized Routes

In [ ]:
# Solve the problem
solver_response = client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

**Notice** that this solution leverages the fact that vehicle 1 is the only vehicle with the ability to perform both service type 1 and service type 2.  In addition, vehicle 0 and vehicle 2 also serve the locations they are suited to service and minimize the time taken along these routes.

In [ ]:
vehicle_colors = ["red", "green", "blue"]
map_vehicle_routes(location_coordinates_df, solver_resp, vehicle_colors).show()

## License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.